<a href="https://colab.research.google.com/github/akalpit23/Explainable-techniques/blob/main/Assignment_8/XAI_in_LLM_Assignment_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment #08



## Akalpit Dawkhar

### **XAI in LLMs**

### Overview